In [1]:
%pip install xgboost

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 222.6 kB/s eta 0:09:22
   ---------------------------------------- 0.0/124.9 MB 330.3 kB/s eta 0:06:19
   ---------------------------------------- 0.1/124.9 MB 365.7 kB/s eta 0:05:42
   ---------------------------------------- 0.1/124.9 MB 655.8 kB/s eta 0:03:11
   ---------------------------------------- 0.6/124.9 MB 2.3 MB/s eta 0:00:56
   ---------------------------------------- 1.1/124.9 MB 3.9 MB/s eta 0:00:32
   ---------------------------------------- 1.5/124.9 MB 4.5 MB/s eta 0:00:28
    --------------------------------------- 2.1/124.9 MB 5.5 MB/s eta 0:00:23
    --------------------------------------- 2.8/124.9 MB 6.6 MB/s eta 0:00:19
   - -------------------------------------- 3.3/124.9 MB 7.1 MB/s eta 0:00:

In [9]:
import pandas as pd

df = pd.read_csv(r"E:\bert_embeddings_dataset.csv")
print(df.shape)

(25000, 12)


In [25]:
import numpy as np
import re

# Function to parse and convert embeddings
def parse_embedding(embedding_str):
    if isinstance(embedding_str, str):  # Check if stored as string
        embedding_str = re.sub(r"\s+", ",", embedding_str.strip("[]"))  # Add commas between numbers
        embedding_list = np.fromstring(embedding_str, sep=",", dtype=np.float32)  # Convert to array
        return embedding_list
    return embedding_str  # If it's already an array, return as is

# Apply the fix to the DataFrame
df["BERT_Embedding"] = df["BERT_Embedding"].apply(parse_embedding)
df.head()

,User ID,Username,Tweet,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,Location,Created At,Hashtags,BERT_Embedding
0,223101,cpierce,song local in friend exist third capital low r...,57,4,76,False,1,South Amanda,2020-09-19 23:42:03,probably value whole sense PM,"[-6.6149917, -6.579562, -6.54237]"
1,206706,breanna13,along business speech standard sound above,34,5,6589,True,0,East Carlamouth,2022-07-05 17:12:28,thought health,"[-6.1911435, -6.149633, -6.138038]"
2,142341,alyssa30,situation drop mouth fill note laugh,81,4,6875,False,1,South Mauriceville,2022-06-28 02:19:48,end oil field sit almost,"[-6.189041, -6.1469626, -6.122707]"
3,562269,shellyerickson,institution student beautiful building five ag...,78,3,9702,False,0,Jacobshire,2020-02-23 18:56:02,PM anything dog begin,"[-7.094418, -7.088888, -7.0139575]"
4,774188,careyjerry,animal prevent rather hundred money field prev...,91,3,9971,False,0,Michaelville,2023-04-10 18:08:35,NaN,"[-7.011471, -7.0120344, -6.9409986]"


In [37]:
from xgboost import XGBClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Extract features and target
df["BERT_Embedding"] = df["BERT_Embedding"].apply(lambda x: np.array(x, dtype=np.float32))
X = np.vstack(df["BERT_Embedding"])
X = np.hstack((X, df[['Retweet Count', 'Mention Count', 'Follower Count', 'Verified']].values))
y = df["Bot Label"]

print(X)
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost Model
model = XGBClassifier(n_estimators=5000, max_depth=25)
model.fit(X_train, y_train)

# Evaluate Model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")

[[-6.614991664886475 -6.579562187194824 -6.542369842529297 ... 4 76 False]
 [-6.19114351272583 -6.149632930755615 -6.138038158416748 ... 5 6589 True]
 [-6.1890411376953125 -6.146962642669678 -6.122706890106201 ... 4 6875
  False]
 ...
 [-6.9832611083984375 -6.956727981567383 -6.851818561553955 ... 0 9402
  False]
 [-6.167236328125 -6.14614200592041 -6.106257438659668 ... 4 7050 False]
 [-6.460955619812012 -6.380080699920654 -6.379101753234863 ... 4 5979
  False]]
Model Accuracy: 0.5130
